In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
data1 = pd.read_csv('/content/sensor_data_nRF_IMU_1_19092025_164551_not_labelled.csv')
data2 = pd.read_csv('/content/sensor_data_nRF_IMU_2_19092025_164551_labelled.csv')

In [ ]:
data1.head()

,TimeStamp,Ax,Ay,Az,Gx,Gy,Gz,Mx,My,Mz
0,16:46:12,9.75,0.90,0.88,0.12,0.18,0.00,14.0,-67.0,-44.0
1,16:46:12,9.76,0.92,0.86,0.12,0.18,-0.12,15.0,-67.0,-42.0
2,16:46:12,9.77,0.91,0.87,0.18,0.18,-0.12,14.0,-65.0,-43.0
3,16:46:12,9.77,0.90,0.88,0.12,0.24,-0.12,14.0,-68.0,-45.0
4,16:46:12,9.78,0.91,0.87,0.18,0.06,-0.06,15.0,-68.0,-39.0


In [ ]:
data2.head()

,TimeStamp,Ax,Ay,Az,Gx,Gy,Gz,Mx,My,Mz,Position,Unnamed: 11
0,16:55:31,-9.20,1.57,1.10,26.61,2.81,-12.08,21,32,-92,NaN,NaN
1,16:55:31,-9.36,1.38,1.25,28.99,-12.08,-14.77,24,33,-88,NaN,NaN
2,16:55:31,-9.37,1.18,1.40,39.92,-15.20,-10.93,26,31,-90,NaN,NaN
3,16:55:31,-9.31,1.37,1.66,39.12,-11.78,-2.44,29,32,-90,NaN,NaN
4,16:55:31,-9.46,1.58,1.74,23.99,-10.01,1.71,29,29,-92,NaN,NaN


In [ ]:
data2 = data2.drop('Unnamed: 11', axis=1)
data2.head()

,TimeStamp,Ax,Ay,Az,Gx,Gy,Gz,Mx,My,Mz,Position
0,16:55:31,-9.20,1.57,1.10,26.61,2.81,-12.08,21,32,-92,NaN
1,16:55:31,-9.36,1.38,1.25,28.99,-12.08,-14.77,24,33,-88,NaN
2,16:55:31,-9.37,1.18,1.40,39.92,-15.20,-10.93,26,31,-90,NaN
3,16:55:31,-9.31,1.37,1.66,39.12,-11.78,-2.44,29,32,-90,NaN
4,16:55:31,-9.46,1.58,1.74,23.99,-10.01,1.71,29,29,-92,NaN


In [ ]:
data2.dropna(inplace=True)
data2.head()

,TimeStamp,Ax,Ay,Az,Gx,Gy,Gz,Mx,My,Mz,Position
195,16:55:54,-8.99,1.25,1.24,-19.41,7.63,-5.55,37,28,-92,1.0
196,16:55:54,-10.44,1.07,2.00,-17.70,9.58,15.08,35,31,-88,1.0
197,16:55:54,-10.47,1.48,0.36,-139.34,2.01,-9.64,28,30,-92,1.0
198,16:55:54,-9.37,1.57,-0.37,-3.54,48.46,-33.57,25,30,-89,1.0
199,16:55:54,-12.47,-0.49,0.39,-57.19,-78.55,-24.35,25,29,-93,1.0


In [ ]:
data = pd.merge(data1, data2, on=['TimeStamp'])
data.head()

,TimeStamp,Ax_x,Ay_x,Az_x,Gx_x,Gy_x,Gz_x,Mx_x,My_x,Mz_x,Ax_y,Ay_y,Az_y,Gx_y,Gy_y,Gz_y,Mx_y,My_y,Mz_y,Position
0,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-8.99,1.25,1.24,-19.41,7.63,-5.55,37,28,-92,1.0
1,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-10.44,1.07,2.00,-17.70,9.58,15.08,35,31,-88,1.0
2,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-10.47,1.48,0.36,-139.34,2.01,-9.64,28,30,-92,1.0
3,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-9.37,1.57,-0.37,-3.54,48.46,-33.57,25,30,-89,1.0
4,16:55:54,9.74,0.77,-0.9,0.37,-5.13,5.25,12.0,-79.0,-74.0,-12.47,-0.49,0.39,-57.19,-78.55,-24.35,25,29,-93,1.0


In [ ]:
data.to_csv('19092025_labelled.csv', index=False)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15081 entries, 0 to 15080
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TimeStamp  15081 non-null  object 
 1   Ax_x       15081 non-null  float64
 2   Ay_x       15081 non-null  float64
 3   Az_x       15081 non-null  float64
 4   Gx_x       15081 non-null  float64
 5   Gy_x       15081 non-null  float64
 6   Gz_x       15081 non-null  float64
 7   Mx_x       15081 non-null  float64
 8   My_x       15081 non-null  float64
 9   Mz_x       15081 non-null  float64
 10  Ax_y       15081 non-null  float64
 11  Ay_y       15081 non-null  float64
 12  Az_y       15081 non-null  float64
 13  Gx_y       15081 non-null  float64
 14  Gy_y       15081 non-null  float64
 15  Gz_y       15081 non-null  float64
 16  Mx_y       15081 non-null  int64  
 17  My_y       15081 non-null  int64  
 18  Mz_y       15081 non-null  int64  
 19  Position   15081 non-null  float64
dtypes: flo

In [ ]:
X = data.drop(['Position','TimeStamp'], axis=1).values
y = data['Position'].values

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
y

array([ 0,  0,  0, ..., 11, 11, 11])

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
def create_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:(i + seq_len)])
        ys.append(y[i + seq_len])
    return np.array(Xs), np.array(ys)

In [ ]:
def create_lstm_model(
    sequence_length=50,
    num_layers=2,
    lstm_units_1=128,
    lstm_units_2=64,
    dropout_rate=0.3,
    dense_units=64,
    activation='relu',
    optimizer='adam',
    loss='sparse_categorical_crossentropy'
):
    model = Sequential()
    model.add(Input(shape=(sequence_length, X.shape[1])))
    model.add(LSTM(lstm_units_1, return_sequences=(num_layers > 1)))
    model.add(Dropout(dropout_rate))
    if num_layers > 1:
        model.add(LSTM(lstm_units_2))
        model.add(Dropout(dropout_rate))
    model.add(Dense(dense_units, activation=activation))
    model.add(Dense(len(np.unique(y)), activation='softmax'))
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

In [ ]:
param_grid = {
    'model__sequence_length': [40, 50, 60],
    'model__num_layers': [1, 2],
    'model__lstm_units_1': [64, 128],
    'model__lstm_units_2': [32, 64],
    'model__dropout_rate': [0.3, 0.5],
    'model__dense_units': [32, 64],
    'model__activation': ['relu', 'tanh'],
    'model__optimizer': ['adam', 'rmsprop'],
    'model__loss': ['sparse_categorical_crossentropy', 'categorical_crossentropy'],
    'batch_size': [32, 64],
    'epochs': [15, 30]
}

In [ ]:
def build_sequences_for_grid(sequence_length):
    X_seq, y_seq = create_sequences(X, y, sequence_length)
    return train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [ ]:
best_score = 0
best_params = None

In [ ]:
for sequence_length in param_grid['model__sequence_length']:
    X_seq, y_seq = create_sequences(X, y, sequence_length)
    X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
    grid = GridSearchCV(
        KerasClassifier(model=create_lstm_model, verbose=1),
        param_grid,
        scoring='accuracy',
        cv=3,
        n_jobs=1
    )
    grid_result = grid.fit(X_train, y_train)
    print(f"Best score for sequence_length={sequence_length}: {grid_result.best_score_}")
    print(f"Best params: {grid_result.best_params_}")
    if grid_result.best_score_ > best_score:
        best_score = grid_result.best_score_
        best_params = grid_result.best_params_

Epoch 1/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.3059 - loss: 1.9736
Epoch 2/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.5082 - loss: 1.3564
Epoch 3/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.6005 - loss: 1.0958
Epoch 4/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.6335 - loss: 1.0067
Epoch 5/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.6818 - loss: 0.8780
Epoch 6/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.7341 - loss: 0.7591
Epoch 7/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.7416 - loss: 0.7153
Epoch 8/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.7696 - loss: 0.6365
Epoch 9/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.7706 - loss: 0.6314
Epoch 10/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.7924 - loss: 0.5777
Epoch 11/15
251/251 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.8089 - loss: 0.5294
Epoch 12/15
251/251 ━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [2]:
print(f"Overall Best score: {best_score}")
print(f"Overall Best Parameters: {best_params}")

Overall Best score: 0.98
Overall Best Parameters: {'model__sequence_length': 50, 'model__num_layers': 2, 'model__lstm_units_1': 128, 'model__lstm_units_2': 64, 'model__dropout_rate': 0.3, 'model__dense_units': 64, 'model__activation': 'relu', 'model__optimizer': 'adam', 'model__loss': 'sparse_categorical_crossentropy', 'batch_size': 64, 'epochs': 30}


In [3]:
print("Thank You")

Thank You
